#데이터과학특론
2020-2학기 구정현

In [1]:
!git clone https://github.com/picaku9/2020_2_data_final.git

Cloning into '2020_2_data_final'...
remote: Enumerating objects: 494, done.
remote: Counting objects: 100% (494/494), done.
remote: Compressing objects: 100% (395/395), done.
remote: Total 494 (delta 86), reused 456 (delta 61), pack-reused 0
Receiving objects: 100% (494/494), 5.14 MiB | 16.59 MiB/s, done.
Resolving deltas: 100% (86/86), done.


In [2]:
pip install singleton_decorator

  Created wheel for singleton-decorator: filename=singleton_decorator-1.0.0-cp36-none-any.whl size=3124 sha256=db2441dceb7af0322c912356905236e9f6d4c271261d9aa1a7b97ed9e9137b28
  Stored in directory: /root/.cache/pip/wheels/ce/43/87/9c4d65e727c32931aca54674ccf4f204e664306fbb507bcbd2
Successfully built singleton-decorator


In [3]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [4]:
pip install git+https://github.com/tensorflow/cleverhans.git#egg=cleverhans

  Cloning https://github.com/tensorflow/cleverhans.git to /tmp/pip-install-c540bdd3/cleverhans
  Running command git clone -q https://github.com/tensorflow/cleverhans.git /tmp/pip-install-c540bdd3/cleverhans
     |████████████████████████████████| 163kB 6.7MB/s 
     |████████████████████████████████| 51kB 6.5MB/s 
  Created wheel for cleverhans: filename=cleverhans-3.0.1-cp36-none-any.whl size=262573 sha256=84ff6ae0618c418e61a84624f35952d5896b35722ddff103a110daaf994a2566
  Stored in directory: /tmp/pip-ephem-wheel-cache-muzxlf3k/wheels/6e/59/ec/723a6f654aaf62c8c40f0f0850fdf71a4948598697f56c3bfa
Successfully built cleverhans


#Available

##cleverhans
###cleverhans_tutorial
1. mnist_tutorial_jsma.py

2. mnist_tutorial_blackbox.py

<li>Test accuracy of oracle on adversarial examples generated using the substitute: 0.565989847715736


##FAIL - 수정사항
###jsma.script.py


dataset.py -> from sklearn.model_selection import StratifiedShuffleSplit

Drebin.py ->  scores = model_selection.cross_val_score(clf, features, labels, cv=5)
from sklearn import svm, model_selection

spsa.py -> delete addons

ModelBasicCNN
jsma_script.py -> from cleverhans.model_zoo.basic_cnn import ModelBasicCNN

import pdb; pdb.set_trace()

model.py -> tf.enable_eager_execution()

In [5]:
%cd /content/2020_2_data_final/research

/content/2020_2_data_final/research


In [6]:
%rm /usr/local/lib/python3.6/dist-packages/cleverhans/attacks/spsa.py
%mv /content/2020_2_data_final/spsa.py /usr/local/lib/python3.6/dist-packages/cleverhans/attacks/spsa.py

In [7]:
%rm /usr/local/lib/python3.6/dist-packages/cleverhans/attacks/attack.py
%mv /content/2020_2_data_final/attack.py /usr/local/lib/python3.6/dist-packages/cleverhans/attacks/attack.py

In [8]:
#!python mnist_blackbox.py
#!python mnist_tutorial_jsma.py
#!python jsma_cleverhans.py
#!python jsma_experiments.py
#!python stingray_experiments.py
#!python jsma_script.py

In [9]:
%cd /content/2020_2_data_final/fail

/content/2020_2_data_final/fail


In [10]:
!python drebin_script.py

Traceback (most recent call last):
  File "drebin_script.py", line 7, in <module>
    from examples.stingray_experiments import StingRayUnconstrainedExperiment
ModuleNotFoundError: No module named 'examples.stingray_experiments'


In [22]:
import numpy as np
import os
import sys

from testbed import testbed
from jsma_experiments import JSMAUnconstrainedExperiment
from stingray_experiments import StingRayUnconstrainedExperiment
from cleverhans.utils_mnist import data_mnist
from cleverhans.utils_tf import model_eval, model_argmax, model_train
from dataset import Dataset
import tensorflow as tf

from cleverhans.loss import LossCrossEntropy
from cleverhans.utils_mnist import data_mnist
from cleverhans.utils_tf import train, model_eval, model_argmax
from cleverhans.model_zoo.basic_cnn import ModelBasicCNN


def one_hot_encoder(ys, nb_classes):
    ohe_ys = []
    for y in ys:
        ohe_y = np.zeros(nb_classes)
        ohe_y[y] = 1

        ohe_ys.append(ohe_y)

    return np.array(ohe_ys)

class CNNModel:

    def __init__(self,scope):
        # Define input TF placeholder
        self.scope = scope
        self.reset()

    def reset(self):
        with tf.variable_scope(self.scope, reuse=tf.AUTO_REUSE):
            self.sess = tf.get_default_session()
            # Define input TF placeholder
            self.x = tf.placeholder(tf.float32, shape=(None, 28, 28, 1))
            self.y = tf.placeholder(tf.float32, shape=(None, 10))
            self.model = ModelBasicCNN(self.scope, 10, 64)
            self.preds = self.model.get_logits(self.x)
            self.loss = LossCrossEntropy(self.model, smoothing=0.1)

    def train(self, dataset):
        train_params = {
            'nb_epochs': 1,
            'batch_size': 32,
            'learning_rate': 1e-2
        }
        with tf.variable_scope(self.scope,reuse=tf.AUTO_REUSE):
            self.sess.run(tf.global_variables_initializer())
            train(self.sess, self.loss, self.x, self.y, dataset.x, dataset.y, args=train_params)

    def test(self, dataset):
        with tf.variable_scope(self.scope, reuse=tf.AUTO_REUSE):
            eval_params = {'batch_size': 32}
            accuracy = model_eval(self.sess, self.x, self.y, self.preds, dataset.x, dataset.y, args=eval_params)
            print('Test accuracy on legitimate test examples: {0}'.format(accuracy))
            return {'accuracy': accuracy}

    def tf(self):
        return self.model


if __name__ == '__main__':
        #with testbed() as tb:
        tb = testbed()
        with tb:
          x_train, y_train, x_test, y_test = data_mnist()
          sess = tf.InteractiveSession()

          victim = CNNModel('victim')
          surrogate = CNNModel('surrogate')

          mnist = Dataset(np.append(x_train, x_test, axis=0), np.append(y_train, y_test, axis=0), train_pct=0.1)
          tb.register_dataset(mnist)
          tb.register_victim(victim)
          tb.register_surrogate(surrogate)

          e1 = JSMAUnconstrainedExperiment(['some_label'])
          #e2 = JSMALimitedFeatureKnowledgeExperiment(border=4)
          #e2 =  StingRayUnconstrainedExperiment(['some_label2'])
          #tb.register_experiment(e2)
          tb.run_experiments(runs=10, out_dir=os.path.join('output', 'jsma-FAIL'))


/usr/local/lib/python3.6/dist-packages/cleverhans/utils_mnist.py:32: UserWarning: cleverhans.utils_mnist is deprecrated and will be removed on or after 2019-03-26. Switch to cleverhans.dataset instead.
  warnings.warn(utils_mnist_warning)
/tensorflow-1.15.2/python3.6/tensorflow_core/python/client/session.py:1750: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


Using default preprocessor.
Using default preprocessor inverse.
No craft function set.
Using default postporcessor.
No apply function set.
Using default target picker (i.e., indiscriminate attack).


/usr/local/lib/python3.6/dist-packages/cleverhans/utils_tf.py:112: UserWarning: This function is deprecated and will be removed on or after 2019-04-05. Switch to cleverhans.train.train.
  warnings.warn("This function is deprecated and will be removed on or after"
/usr/local/lib/python3.6/dist-packages/cleverhans/loss.py:275: UserWarning: LossCrossEntropy is deprecated, switch to CrossEntropy. LossCrossEntropy may be removed on or after 2019-03-06.
  warnings.warn("LossCrossEntropy is deprecated, switch to "
[INFO 2020-12-17 20:42:01,053 cleverhans] Epoch 0 took 19.374618530273438 seconds
[INFO 2020-12-17 20:42:01,054 cleverhans] Completed model training.
/usr/local/lib/python3.6/dist-packages/cleverhans/attacks/attack.py:36: UserWarning: Argument back to attack constructors is not needed anymore and will be removed on or after 2019-03-26. All attacks are implemented using TensorFlow.
  warnings.warn("Argument back to attack constructors is not needed"
[INFO 2020-12-17 20:42:06,744 clev

Test accuracy on legitimate test examples: 0.9579285714285715


/usr/local/lib/python3.6/dist-packages/cleverhans/compat.py:22: UserWarning: <function reduce_sum_v1 at 0x7f54e206fb70> is deprecated. Switch to calling the equivalent function in tensorflow.  This function was originally needed as a compatibility layer for old versions of tensorflow,  but support for those versions has now been dropped.
  warnings.warn(str(f) + " is deprecated. Switch to calling the equivalent function in tensorflow. "
/usr/local/lib/python3.6/dist-packages/cleverhans/compat.py:22: UserWarning: <function reduce_max_v1 at 0x7f54e2073a60> is deprecated. Switch to calling the equivalent function in tensorflow.  This function was originally needed as a compatibility layer for old versions of tensorflow,  but support for those versions has now been dropped.
  warnings.warn(str(f) + " is deprecated. Switch to calling the equivalent function in tensorflow. "
/usr/local/lib/python3.6/dist-packages/cleverhans/compat.py:22: UserWarning: <function reduce_any_v1 at 0x7f54e2078048

...

Test accuracy on legitimate test examples: 0.9600714285714286
Test accuracy on legitimate test examples: 0.0
Test accuracy on legitimate test examples: 0.0
some_label
Classifier: Surrogate.
Performance: {'accuracy': 0.9639285714285715}
Target info: {'target_instance': array([ True,  True,  True,  True,  True,  True, False,  True,  True,
        True]), 'source_class': 0, 'target_class': 5}
Classifier: Victim.
Performance: {'accuracy': 0.9600714285714286}
Attack performance: {'surrogate_acc': {'accuracy': 0.0}, 'victim_acc': {'accuracy': 0.0}}


/usr/local/lib/python3.6/dist-packages/cleverhans/utils_tf.py:112: UserWarning: This function is deprecated and will be removed on or after 2019-04-05. Switch to cleverhans.train.train.
  warnings.warn("This function is deprecated and will be removed on or after"
/usr/local/lib/python3.6/dist-packages/cleverhans/loss.py:275: UserWarning: LossCrossEntropy is deprecated, switch to CrossEntropy. LossCrossEntropy may be removed on or after 2019-03-06.
  warnings.warn("LossCrossEntropy is deprecated, switch to "
[INFO 2020-12-17 20:50:15,111 cleverhans] Epoch 0 took 19.867311239242554 seconds
[INFO 2020-12-17 20:50:15,112 cleverhans] Completed model training.
/usr/local/lib/python3.6/dist-packages/cleverhans/attacks/attack.py:36: UserWarning: Argument back to attack constructors is not needed anymore and will be removed on or after 2019-03-26. All attacks are implemented using TensorFlow.
  warnings.warn("Argument back to attack constructors is not needed"
[INFO 2020-12-17 20:50:21,046 clev

Test accuracy on legitimate test examples: 0.9517857142857142


/usr/local/lib/python3.6/dist-packages/cleverhans/compat.py:22: UserWarning: <function reduce_sum_v1 at 0x7f54e206fb70> is deprecated. Switch to calling the equivalent function in tensorflow.  This function was originally needed as a compatibility layer for old versions of tensorflow,  but support for those versions has now been dropped.
  warnings.warn(str(f) + " is deprecated. Switch to calling the equivalent function in tensorflow. "
/usr/local/lib/python3.6/dist-packages/cleverhans/compat.py:22: UserWarning: <function reduce_max_v1 at 0x7f54e2073a60> is deprecated. Switch to calling the equivalent function in tensorflow.  This function was originally needed as a compatibility layer for old versions of tensorflow,  but support for those versions has now been dropped.
  warnings.warn(str(f) + " is deprecated. Switch to calling the equivalent function in tensorflow. "
/usr/local/lib/python3.6/dist-packages/cleverhans/compat.py:22: UserWarning: <function reduce_any_v1 at 0x7f54e2078048

Test accuracy on legitimate test examples: 0.951
Test accuracy on legitimate test examples: 0.0
Test accuracy on legitimate test examples: 0.0
some_label
Classifier: Surrogate.
Performance: {'accuracy': 0.9517857142857142}
Target info: {'target_instance': array([ True,  True,  True,  True, False,  True,  True,  True,  True,
        True]), 'source_class': 0, 'target_class': 5}
Classifier: Victim.
Performance: {'accuracy': 0.951}
Attack performance: {'surrogate_acc': {'accuracy': 0.0}, 'victim_acc': {'accuracy': 0.0}}
